In [2]:
import argparse
import random
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas
import torch
from braindecode import EEGRegressor
from braindecode.models.util import get_output_shape
from braindecode.util import np_to_var
from skorch.callbacks import TensorBoard, Checkpoint

from Interpretation.interpretation import get_corr_coef
from Training.CorrelationMonitor1D import CorrelationMonitor1D
from Training.train import get_model, get_writer
from data.pre_processing import get_num_of_channels, Data
from global_config import home, input_time_length, cuda, random_seed
from models.DoubleModel import DoubleModel
from models.Model import load_model

%matplotlib notebook

parser = argparse.ArgumentParser()
parser.add_argument('--variable', default=1, type=int)

args = parser.parse_args()
dilation = [None]
kernel_size = [3, 3, 3, 3]
trajectory_index = args.variable
low_pass = False
shift_by = None
lr = 0.001
shift = True
max_train_epochs = 100
if trajectory_index == 0:
    model_string = f'sbp0_sm_vel'
    variable = 'vel'
else:
    model_string = 'sbp0_sm_absVel'
    variable = 'absVel'
all_xs = []
all_ys = []
all_zs = []
corr_coefs_full = []
corr_coefs_hp = []
for patient_index in range(1, 13):
    input_channels = get_num_of_channels(home + f'/previous_work/P{patient_index}_data.mat')
    # model, changed_model_full, model_name = get_model(input_channels, input_time_length,
    #                                                   dilations=dilation[0],
    #                                                   kernel_sizes=kernel_size, padding=False)
    changed_model_full = load_model(f'/models/saved_models/lr_0.001/sbp1_sm_{variable}_k_3333/sbp1_sm_{variable}_k_3333_p_{patient_index}/best_model_split_0')
    # _, changed_model_hp, _ = get_model(input_channels, input_time_length,
    #                                    dilations=dilation[0],
    #                                    kernel_sizes=kernel_size, padding=False)
    changed_model_hp = load_model(f'/models/saved_models/lr_0.001/sbp1_hps_{variable}_k_3333/sbp1_hps_{variable}_k_3333_p_{patient_index}/best_model_split_0')
    model_name = 'k3_d3'
    n_preds_per_input = get_output_shape(changed_model_full, input_channels, input_time_length)[1]
    small_window = input_time_length - n_preds_per_input + 1
    if shift_by is None:
        shift_index = int(small_window / 2)
    else:
        shift_index = int((small_window / 2) - shift_by)

    data_full = Data(home + f'/previous_work/P{patient_index}_data.mat', num_of_folds=-1,
                     low_pass=low_pass,
                     trajectory_index=trajectory_index, shift_data=shift, high_pass=False,
                     shift_by=int(shift_index),
                     valid_high_pass=False, low_pass_training=False, double_training=True)
    data_hp = Data(home + f'/previous_work/P{patient_index}_data.mat', num_of_folds=-1,
                   low_pass=low_pass,
                   trajectory_index=trajectory_index, shift_data=shift, high_pass=True,
                   shift_by=int(shift_index),
                   valid_high_pass=False, low_pass_training=False)
    output_dir = f'{model_string}_{model_name}/{model_string}_{model_name}_p_{patient_index}'
    correlation_monitor = CorrelationMonitor1D(input_time_length=input_time_length,
                                               output_dir=output_dir)

    if cuda:
        device = 'cuda'
        changed_model_full = changed_model_full.cuda()
        changed_model_hp = changed_model_hp.cuda()

    else:
        device = 'cpu'

    model = DoubleModel(changed_model_full, changed_model_hp)

    data_hp.cut_input(input_time_length, n_preds_per_input, False)
    data_full.cut_input(input_time_length, n_preds_per_input, False)

2021-03-17 09:14:55,295 DEBUG : Loaded backend nbAgg version unknown.


usage: ipykernel_launcher.py [-h] [--variable VARIABLE]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/m_vys/Library/Jupyter/runtime/kernel-0a016d1a-92f6-439a-a5b2-b699f1e978b2.json
/Users/m_vys/Documents/matfyz/magistr/diplomka/ECoG_decoders/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


SystemExit: 2

In [ ]:
    corr_coef_full = get_corr_coef(data_full.test_set, changed_model_full)
    corr_coef_hp = get_corr_coef(data_hp.test_set, changed_model_hp)
    print('coef_full:', corr_coef_full, 'coef_hp:', corr_coef_hp)
    corr_coefs_full.append(corr_coef_full)
    corr_coefs_hp.append(corr_coef_hp)
    output = f'{home}/results/model_to_true_predictions/{model_string}_{model_name}/valid/graph_patient_{patient_index}_3d.png'
    Path(f'{home}/results/model_to_true_predictions/{model_string}_{model_name}/valid_data/').mkdir(parents=True, exist_ok=True)


In [ ]:
    xs = []
    ys = []
    zs = []
    for input_full, input_hp, correct_out in zip(data_full.test_set.X, data_hp.test_set.X, data_full.test_set.y):
        out_full = changed_model_full.double()(np_to_var(input_full.reshape([1, input_full.shape[0], input_full.shape[1], input_full.shape[2]])).double())
        out_hp = changed_model_hp.double()(np_to_var(input_hp.reshape(1, input_full.shape[0], input_full.shape[1], input_full.shape[2])).double())
        xs.append(out_full)
        ys.append(out_hp)
        zs.append(correct_out.reshape([1, correct_out.shape[0]]))
        for x, y, z in zip(out_full[0], out_hp[0], correct_out):
            string = f'[{float(x)}, {float(y)}, {z}],\n'
        # out_full = out_full.reshape([out_full.shape[1]])
        # out_hp = out_hp.reshape([out_hp.shape[1]])

        print(correct_out.shape)
        fig = plt.figure()

        ax = fig.add_subplot(111, projection='3d')
        ax.scatter3D(out_full, out_hp, correct_out, c=correct_out, cmap='Greens')
        ax.set_xlabel('Full model predictions')
        ax.set_ylabel('High-pass model predictions')
        ax.set_zlabel('True values')
        plt.show()
